# Python - Class Inheritance

- https://openhome.cc/Gossip/Python/Inheritance.html
- http://python3-cookbook.readthedocs.io/zh_CN/latest/c08/p07_calling_method_on_parent_class.html
- http://funhacks.net/explore-python/Class/super.html

## 理解 Python 中 `super()` 和 `__init__()` 方法

`super()`的好處就是可以避免直接使用父類的名字.但是它主要用於多重繼承,[這裡面](http://www.artima.com/weblogs/viewpost.jsp?thread=236275)有很多好玩的東西.如果還不瞭解的話可以看看[官方文檔](https://docs.python.org/2/library/functions.html#super)

In [3]:
class Base(object):
    def __init__(self):
        print("Base created")

class ChildA(Base):
    def __init__(self):
        Base.__init__(self)

class ChildB(Base):
    def __init__(self):
        super(ChildB, self).__init__()

print(ChildA(), ChildB())

Base created
Base created
<__main__.ChildA object at 0x105a85dd8> <__main__.ChildB object at 0x105aa8080>


在Python中，可以進行多重繼承，這個時候要注意搜尋的順序，是從子類別開始，接著是同一階層父類別由左至右搜尋，再至更上層同一階層父類別由左 至右搜尋，直到達到頂層為止。例如：

In [7]:
class A(object):
    def method1(self):
        print('A.method1')
    
    def method2(self):
        print('A.method2')

class B(A):
    def method3(self):
        print('B.method3')

class C(A):
    def method2(self):
        print('C.method2')
    
    def method3(self):
        print('C.method3')

class D(B, C):
    def method4(self):
        print('D.method4')

        
d = D()
d.method4() # 在 D 找到，D.method4
d.method3() # 以 D->B 順序找到，B.method3
d.method2() # 以 D->B->C 順序找到，C.method2
d.method1() # 以 D->B->C->A 順序找到，A.method1

D.method4
B.method3
C.method2
A.method1


---

## Advanced Inheritance

http://funhacks.net/explore-python/Class/super.html

```python
class Base(object):
    def __init__(self):
        print("enter Base")
        print("leave Base")

class A(Base):
    def __init__(self):
        print("enter A")
        super(A, self).__init__()
        print("leave A")

class B(Base):
    def __init__(self):
        print("enter B")
        super(B, self).__init__()
        print("leave B")

class C(A, B):
    def __init__(self):
        print("enter C")
        super(C, self).__init__()
        print("leave C")
```

其中，Base 是父類，A, B 繼承自 Base, C 繼承自 A, B，它們的繼承關係是一個典型的『菱形繼承』，如下：

```
      Base
      /  \
     /    \
    A      B
     \    /
      \  /
       C
```

現在，讓我們看一下使用：

```python
>>> c = C()
enter C
enter A
enter B
enter Base
leave Base
leave B
leave A
leave C
```

如果你認為 `super` 代表『調用父類的方法』，那你很可能會疑惑為什麼 enter A 的下一句不是 enter Base 而是 enter B。原因是，**`super` 和父類沒有實質性的關聯**，現在讓我們搞清 `super` 是怎麼運作的。

事實上，對於你定義的每一個類，Python 會計算出一個**方法解析順序（Method Resolution Order, MRO）列表**，**它代表了類繼承的順序**，我們可以使用下面的方式獲得某個類的 MRO 列表：

```python
>>> C.mro()   # or C.__mro__ or C().__class__.mro()
[__main__.C, __main__.A, __main__.B, __main__.Base, object]
```

那這個 MRO 列表的順序是怎麼定的呢，它是通過一個 [C3 線性化算法](https://www.python.org/download/releases/2.3/mro/)來實現的，這裡我們就不去深究這個算法了，感興趣的讀者可以自己去瞭解一下，總的來說，一個類的 MRO 列表就是合併所有父類的 MRO 列表，並遵循以下三條原則：

-   子類永遠在父類前面
-   如果有多個父類，會根據它們在列表中的順序被檢查
-   如果對下一個類存在兩個合法的選擇，選擇第一個父類

當你使用 `super(cls, inst)` 時，Python 會在 inst 的 MRO 列表上搜索 cls 的下一個類。

現在，讓我們回到前面的例子。

首先看類 C 的 `__init__` 方法：

```python
super(C, self).__init__()
```

這裡的 self 是當前 C 的實例，self.**class**.mro() 結果是：

```
[__main__.C, __main__.A, __main__.B, __main__.Base, object]
```

可以看到，C 的下一個類是 A，於是，跳到了 A 的 `__init__`，這時會打印出 enter A，並執行下面一行代碼：

```python
super(A, self).__init__()
```

注意，這裡的 self 也是當前 C 的實例，MRO 列表跟上面是一樣的，搜索 A 在 MRO 中的下一個類，發現是 B，於是，跳到了 B 的 `__init__`，這時會打印出 enter B，而不是 enter Base。